In [5]:
import os
import re
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt



In [6]:
class PRPDMetrics():

    def __init__(self, PRPD: np.ndarray):
        """
        Inicializa a classe com um array.

        Args:
            PRPD(ndarray): Array NumPy contendo valores de amplitude e fase.
        """

        if not isinstance(PRPD, np.ndarray): # Validações de entrada
            raise TypeError("PRPD deve ser um array NumPy.")
        if PRPD.size == 0:
            PRPD = np.zeros(2,2)
        
        self.PRPD = PRPD   
        self._minimum = None # prefixo _ indica que são variáveis internas (convenção Python)
        self._maximum = None
        self._elements = None
        self._mean = None
        self._Firstquartile = None
        self._Secondquartile = None
        self._Thirdquartile = None
        self._Assymetry = None
        self._Kurtosis = None
        self._dataframe = None

    @property # permite acessar o método como se fosse um atributo (sem parênteses)
    def minimum(self) -> float:
        """Calcula e retorna o valor minimo dos vetores de amplitudes e fases"""
        if self._minimum is None:
            self._minimum = np.min(self.PRPD, axis=1)
        return self._minimum
    
    @property
    def maximum(self) -> float:
        """Calcula e retorna o valor maximo dos vetores de amplitude e fase"""
        if self._maximum is None:
            self._maximum = np.max(self.PRPD, axis=1)
        return self._maximum
    
    @property
    def elements(self) -> float:
        """Calcula e retorna a quantidade de pulsos"""
        if self._elements is None:
            self._elements = self.PRPD.shape[1]
        return self._elements
    
    @property
    def mean(self) -> float:
        """Calcula e retorna a media dos amplitudes e fases"""
        if self._mean is None:
            self._mean = np.mean(self.PRPD, axis=1)
        return self._mean
    
    @property
    def firstquartile(self) -> float:
        """Calcula e retorna o primeiro quartil das amplitudes e fases"""
        if self._Firstquartile is None:
            self._Firstquartile = np.percentile(self.PRPD, 25, axis=1)
        return self._Firstquartile
    
    @property
    def secondquartile(self) -> float:
        """Calcula e retorna o segundo quartil das amplitudes e fases"""
        if self._Secondquartile is None:
            self._Secondquartile = np.percentile(self.PRPD, 50, axis=1)
        return self._Secondquartile
    
    @property
    def thirdquartile(self) -> float:
        """Calcula e retorna o terceiro quartil das amplitudes e fases"""
        if self._Thirdquartile is None:
            self._Thirdquartile = np.percentile(self.PRPD, 75, axis=1)
        return self._Thirdquartile
    
    @property
    def assymetry(self) -> float:
        """Calcula e retorna o assimetria das amplitudes e fases"""
        if self._Assymetry is None:
            self._Assymetry = scipy.stats.skew(self.PRPD, axis=1)
        return self._Assymetry
    
    @property
    def kurtosis(self) -> float:
        """Calcula e retorna a curtose das amplitudes e fases"""
        if self._Kurtosis is None:
            self._Kurtosis = scipy.stats.kurtosis(self.PRPD, axis=1)
        return self._Kurtosis  
    
    @property
    def dataframe(self) -> pd.DataFrame:
        """Retorna um DataFrame com todas as métricas calculadas"""
        if self._dataframe is None:
            self._dataframe = pd.DataFrame({
                'Minimum': self.minimum,
                'Maximum': self.maximum,
                'Elements': self.elements,
                'Mean': self.mean,
                'First Quartile': self.firstquartile,
                'Second Quartile': self.secondquartile,
                'Third Quartile': self.thirdquartile,
                'Assymetry': self.assymetry,
                'Kurtosis': self.kurtosis}, 
                index = ['Amplitude', 'Fase'])
        return self._dataframe
            
        
    

In [7]:
def extrair_numero(s):
    matches = list(re.finditer(r'\d+', s))
    if matches:
        return int(matches[-1].group())  # Pega a última correspondência
    else:
        return float('inf')  # Retorna infinito se não houver número




def PRPDExtraction(Diretório_entrada: str, Diretório_saida: str) -> None:
    """
    Extrai as características de um arquivo .mat contendo dados PRPD e salva em um arquivo .csv
    pronto para jogar como entrada em um classificador.

    Args:
        Diretório_entrada(str): Diretório onde estão os arquivos .mat.
        Diretório_saida(str): Diretório onde serão salvos os arquivos .csv.
    """

    #Lista os arquivos no diretório de entrada e define um DataFrame vazio
    Arquivos = os.listdir(Diretório_entrada)
    print(f'Diretorio sem organizar: {Arquivos}')
    df = pd.DataFrame()
    k = 0


    # Ordenar numericamente com base na parte numérica
    arquivos_ordenados = sorted(Arquivos, key=extrair_numero)
    print(f'Diretorio organizado {arquivos_ordenados}')

    #Define uma lista de indexadores para renomear as linhas do DataFrame:
    Index_amp_sp = ['Mininimum_amp_+', 'Maximum_amp_+', 'Elements_+', 'Mean_amp_+', 'First quartile_amp_+', 'Second quartile_amp_+', 'Third quartile_amp_+', 'Assymetry_amp_+', 'Kurtosis_amp_+']
    Index_fase_sp = ['Mininimum_fase_+', 'Maximum_fase_+', 'Mean_fase_+', 'First quartile_fase_+', 'Second quartile_fase_+', 'Third quartile_fase_+', 'Assymetry_fase_+', 'Kurtosis_fase_+']
    Index_amp_sn = ['Mininimum_amp_-', 'Maximum_amp_-', 'Elements_-', 'Mean_amp_-', 'First quartile_amp_-', 'Second quartile_amp_-', 'Third quartile_amp_-', 'Assymetry_amp_-', 'Kurtosis_amp_-']
    Index_fase_sn = ['Mininimum_fase_-', 'Maximum_fase_-', 'Mean_fase_-', 'First quartile_fase_-', 'Second quartile_fase_-', 'Third quartile_fase_-', 'Assymetry_fase_-', 'Kurtosis_fase_-']
    
    #Criando um dataframe para armazenar todos os atributos em um único dataframe:
    df_geral = pd.DataFrame()

    #Criando uma lista para rotular os PRPDs dentro do dataframe geral
    index_k = []
    
    #Definição do laço para percorrer os arquivos no diretório de entrada
    for i in range(len(arquivos_ordenados)):
        k += 1
        
        if arquivos_ordenados[i].endswith(".mat"):

            #Carrega os dados do arquivo .mat
            dados = scipy.io.loadmat(os.path.join(Diretório_entrada, arquivos_ordenados[i]))

            #Cria um DataFrame com os dados do arquivo .mat
            df = pd.DataFrame(list(dados.values())[-1])

            # Criar a coluna de classe
            df.loc['classe'] = ['positivo' if valor <= 180 else 'negativo' for valor in df.iloc[1, :]]

            # Filtrar os elementos do semi-ciclo positivo
            dados_sp = df.loc[:, df.loc['classe'] == 'positivo'].drop('classe').to_numpy()[:2, :]
            dados_sp = dados_sp.astype(float)  # Garantir que todos os elementos sejam floats

            # Filtrar elementos do semi-ciclo negativo
            dados_sn = df.loc[:, df.loc['classe'] == 'negativo'].drop('classe').to_numpy()[:2, :]
            dados_sn = dados_sn.astype(float)  # Garantir que todos os elementos sejam floats

            # Filtrar os elementos do semi-ciclo positivo
            dados_sp = df.loc[:, df.loc['classe'] == 'positivo'].drop('classe').to_numpy()[:2, :]
            dados_sp = dados_sp.astype(float)  # Garantir que todos os elementos sejam floats

            # Criar a coluna de classe
            df.loc['classe'] = ['positivo' if valor <= 180 else 'negativo' for valor in df.iloc[1, :]]

            # Filtrar elementos do semi-ciclo negativo
            dados_sn = df.loc[:, df.loc['classe'] == 'negativo'].drop('classe').to_numpy()[:2, :]
            dados_sn = dados_sn.astype(float)  # Garantir que todos os elementos sejam floats
            
            #Normalizar os vetores de amplitudes dos semi-ciclos:
            dados_sn[0,:] = dados_sn[0,:]/max(dados_sn[0,:])
            dados_sp[0,:] = dados_sp[0,:]/max(dados_sp[0,:])

            #Aplicando a classe metrics para extrair os atributos:
            metrics_sn = PRPDMetrics(dados_sn)
            metrics_sp = PRPDMetrics(dados_sp)

            #Definindo o nome do arquivo de saída
            nome_arquivo = 'PRPDfeatures_' + arquivos_ordenados[i][:-4] + '.csv'
            Diretório_saida_arq = os.path.join(Diretório_saida, nome_arquivo)

            #Extraindo os dados do Dataframe
            df_sp = metrics_sp.dataframe.T
            df_sn = metrics_sn.dataframe.T

            #Separando os dados de amplitude e fase e renomeando as linhas
            df_sp_amplitude = df_sp['Amplitude']
            df_sp_amplitude.index = Index_amp_sp
            
            df_sn_amplitude = df_sn['Amplitude']
            df_sn_amplitude.index = Index_amp_sn

            #Descarto a linha de elementos pois ja está contida na coluna de fase
            df_sp_fase = df_sp['Fase']
            df_sp_fase.drop('Elements', inplace = True)
            df_sp_fase.index = Index_fase_sp

            df_sn_fase = df_sn['Fase']
            df_sn_fase.drop('Elements', inplace = True)
            df_sn_fase.index = Index_fase_sn

            #Concatenando os dados de amplitude e fase por linha
            df_sp = pd.concat([df_sp_amplitude, df_sp_fase], axis = 0)
            
            #Printando os resultados para avaliar coerencia:
            #print("Semicilo positivo")
            #print(df_sp_amplitude)
            #print('\n')
            #print(df_sp_fase)
            #print('\n')
            #print(df_sp)
            #print('\n')
            
            df_sn = pd.concat([df_sn_amplitude, df_sn_fase], axis = 0)

            #print("Semicilo negativo")
            #print('\n')
            #print(df_sn_amplitude)
            #print('\n')
            #print(df_sn_fase)
            #print('\n')
            #print(df_sn)
            #print('\n')

            #Concatenando o semiciclo positivo e negativo por linha
            df = pd.concat([df_sp, df_sn], axis = 0)
            
            df.to_csv(Diretório_saida_arq, index = True, header = True)
            index_k.append(f'PRPD_{k}')
            

        else:
            continue

        df_geral = pd.concat([df_geral, df], axis = 1)

    
    df_geral.columns = index_k
    df_geral = df_geral.T
    df_geral.to_csv(os.path.join(Diretório_saida, 'PRPDfeatures_geral.csv'), index = True)
    

Diretório_entrada = r"C:\Users\boave\OneDrive\Desktop\Pasta Pessoal - Boaventura Filho\PIBIC\Dados"
Diretório_saida = r"C:\Users\boave\OneDrive\Desktop\Pasta Pessoal - Boaventura Filho\PIBIC\Resultados\Barra_6kv"

PRPDExtraction(Diretório_entrada, Diretório_saida,)

Diretorio sem organizar: ['barra_6kv_condi_1.mat', 'barra_6kv_condi_2.mat', 'barra_6kv_condi_22.mat', 'barra_6kv_condi_6.mat']
Diretorio organizado ['barra_6kv_condi_1.mat', 'barra_6kv_condi_2.mat', 'barra_6kv_condi_6.mat', 'barra_6kv_condi_22.mat']


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\boave\\OneDrive\\Desktop\\Pasta Pessoal - Boaventura Filho\\PIBIC\\Resultados\\Barra_6kv\\PRPDfeatures_geral.csv'

In [ ]:
# Validando os arquivos salvos
arquivos_saida = os.listdir(Diretório_saida)
arquivos_saida = sorted(arquivos_saida, key=extrair_numero)

for arquivo in arquivos_saida:
    df = pd.read_csv(os.path.join(Diretório_saida, arquivo), index_col = 0)
    print(f'Arquivo: {arquivo}')
    print(df)
    print(f'Dimensão do DataFrame: {df.shape}')
    print('\n')



Arquivo: PRPDfeatures_barra_6kv_condi_1.csv
                                 0
Mininimum_amp_+           0.659698
Maximum_amp_+             1.000000
Elements_+               29.000000
Mean_amp_+                0.827449
First quartile_amp_+      0.767976
Second quartile_amp_+     0.806646
Third quartile_amp_+      0.911831
Assymetry_amp_+           0.229791
Kurtosis_amp_+           -0.835876
Mininimum_fase_+         18.125502
Maximum_fase_+           76.367047
Mean_fase_+              43.576051
First quartile_fase_+    24.961132
Second quartile_fase_+   43.053910
Third quartile_fase_+    55.651693
Assymetry_fase_+          0.176098
Kurtosis_fase_+          -1.120394
Mininimum_amp_-           0.614595
Maximum_amp_-             1.000000
Elements_-               23.000000
Mean_amp_-                0.822857
First quartile_amp_-      0.735483
Second quartile_amp_-     0.815676
Third quartile_amp_-      0.912027
Assymetry_amp_-          -0.125541
Kurtosis_amp_-           -0.972562
Mininimum_f